In [1]:
# 云4o配置
from openai import OpenAI
from transformers import set_seed
import json_repair
import time
import random
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cdist


class GPTAgent:
    def __init__(self, model_name, system_prompt, task="", max_new_tokens=512,
                 temperature=0.7, seed=10086, time_sleep_min=10, time_sleep_max=30):
        """
        初始化 GPTAgent
        """
        self.max_new_tokens = max_new_tokens
        self.model_name = model_name
        self.temperature = temperature
        self.model = OpenAI(
            base_url="https://xiaoai.plus/v1",
            api_key="sk-GYvK0yJXWFkz6ADkyJKviUPnOjKoolYTLv1RH4DfQMyUwrTK",
            timeout=300000
        )
        self.system_prompt = system_prompt
        self.seed = seed
        self.task = task
        self.time_sleep_min = time_sleep_min
        self.time_sleep_max = time_sleep_max
        set_seed(self.seed)

    def query(self, prompt, print_prompt=False):
        """
        调用 OpenAI API 并返回结果
        """
        # 限制 prompt 长度
        prompt = prompt[:32768]
        if print_prompt:
            print(f"Prompt: {prompt}")

        # 随机睡眠模拟请求间隔
        sleep_time = random.randint(self.time_sleep_min, self.time_sleep_max)
        # print(f"Sleeping for {sleep_time} seconds...")
        time.sleep(sleep_time)

        try:
            completion = self.model.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt},
                ],
                temperature=self.temperature,
            )

            # 获取 API 响应内容
            res = completion.choices[0].message.content
            if print_prompt:
                print(f"Response: {res}")

            # 修复 JSON 格式并返回
            # return json_repair.loads(res.replace("```json", "").replace("```", ""))
            return res

        except Exception as e:
            print(f"Error during API call: {e}")
            return {"error": str(e)}

In [2]:
# parameter

# # 初始自体非自体
# self = pd.read_csv('../dataset/train_self.csv')
# nonself = pd.read_csv('../dataset/train_nonself.csv')

# # 原始数据集（小样本）
# fewshot = pd.read_csv('../dataset/fewshot.csv')
# fewshot.to_csv("detectors_0.csv",index=False)

# 未知类型
unknown_type = 'A'
Round = 7
# 初始自体非自体
# self = pd.read_csv(f'../dataset/UNSW_NB15/self/self.csv')
# seed = pd.read_csv(f'../dataset/UNSW_NB15/seed_0/{unknown_type}/Round-1/trainset_{unknown_type}_nonself.csv')
# # train_set_unknown = pd.read_csv(f'../dataset/UNSW_NB15/unknown/train/train{unknown_type}.csv')
# # test_set_unknown = pd.read_csv(f'../dataset/test_20/check/nonself/test/test{unknown_type}.csv')

# seed.to_csv("detectors_0.csv",index=False)    

# with open(f"coverage_results_epoch.txt", "w") as f:
#     f.write(f"Unknown type: {unknown_type}\n")
#     f.write(f"Number of self samples: {len(self)}\n")
#     f.write(f"Number of known samples: {len(known)}\n")
#     f.write(f"Number of unknown samples in train_set: {len(train_set_unknown)}\n")
#     f.write(f"Number of unknown samples in test_set: {len(test_set_unknown)}\n")
    
    
# 自体半径
# self_radius = 0.005

# new_detectors = pd.read_csv('generated.csv')

# 变异子数：1->n
some = "5"

# 变异轮数
round = 2

# 数据维度
dimension = "42"


In [3]:
# tool

# # 读取检测器
# def get_detectors(detector_df):
#     rows = detector_df.apply(lambda row: ','.join(map(str, row)), axis=1)
#     detectors = ('[' + '; '.join(rows) + ']').tolist()
#     return detectors

# 读取数据
def get_data(data_df):
    data = data_df.map(str).apply(lambda row: f"[{' '.join(row)}]", axis=1).tolist()
    return data

# # 添加检测半径
# def add_detection_radius(detectors_df, self_df):
#     detector_coords = detectors_df.values
#     self_coords = self_df.values
#     distances = cdist(detector_coords, self_coords)
#     detectors_df['radius'] = distances.min(axis=1) - self_radius
#     detectors_df['radius'] = detectors_df['radius'].clip(lower=0)  # 确保半径非负
#     return detectors_df


# # 评估非自体覆盖率
# def evaluate_nonselfcoverage(detectors_df, nonself_df):
#     detector_coords = detectors_df.values[:,:-1]
#     nonself_coords = nonself_df.values
#     distances = cdist(nonself_coords, detector_coords)
#     radii = detectors_df['radius'].values.reshape(1, -1)
#     covered = (distances <= radii).any(axis=1)
#     covered_count = covered.sum()
    
#     # nonself_df['covered'] = covered
#     # return nonself_df
#     return covered_count
    
# # 评估自体覆盖率
# def evaluate_selfcoverage(detectors_df, self_df):
#     detector_coords = detectors_df.values[:,:-1]
#     self_coords = self_df.values
#     distances = cdist(self_coords, detector_coords)
#     radii = detectors_df['radius'].values.reshape(1, -1)
#     covered = (distances <= radii).any(axis=1)
#     covered_count = covered.sum()
#     return covered_count

# # 小样本生成变异评估
# def evaluate_fewshot(epoch, detectors_df, self, nonself):
#     detectors_df_copy = detectors_df.copy()
#     detectors_radius = add_detection_radius(detectors_df_copy, self)
#     # print(f"round:{epoch}\n")
#     with open(f"coverage_results_epoch.txt", "a") as f:
#         f.write(f"Epoch: {epoch}\n")
#         f.write(f"Number of detectors: {len(detectors_df)}")
#         f.write(f"Number of non-self samples covered by detectors: {evaluate_nonselfcoverage(detectors_radius,nonself)}\n")

# # 未知攻击覆盖变异评估
# def evaluate_unknown(epoch, detectors_df):
#     detectors_df_copy = detectors_df.copy()
#     detectors_radius = add_detection_radius(detectors_df_copy, self)
#     # print(f"round:{epoch}\n")
#     with open(f"coverage_results_epoch.txt", "a") as f:
#         f.write(f"Epoch: {epoch}\n")
#         f.write(f"Number of detectors: {len(detectors_df)}\n")
#         f.write(f"Number of unknown samples in train_set covered by detectors: {evaluate_nonselfcoverage(detectors_radius,train_set_unknown)}\n")
#         f.write(f"Number of unknown samples in test_set covered by detectors: {evaluate_nonselfcoverage(detectors_radius,test_set_unknown)}\n")



In [4]:
agent = GPTAgent(
                model_name="gpt-4o-mini",  # 替换为实际模型名称
                system_prompt="Describe yourself",
                task="Data Synthesis ",
                max_new_tokens=640,
                temperature=0.5,
                seed=42,
                time_sleep_min=1,
                time_sleep_max=3,
            )
prompt = "who are you?"
result = agent.query(prompt, print_prompt=False)
print(result)

I am an AI language model created by OpenAI, designed to assist with a wide range of questions and tasks. My training includes a diverse set of topics, and I'm here to provide information, answer questions, and engage in conversation. My knowledge is based on data available up to October 2023. How can I help you today?


In [3]:
for r in range(Round):
    known = pd.read_csv(f'../dataset/UNSW_NB15/seed_0/{unknown_type}/Round{r+1}/train{unknown_type}_nonself.csv')
    known.to_csv(f"Round{r+1}/detectors_0.csv",index=False)
    for epoch in range(round):
        dataset = pd.read_csv(f"Round{r+1}/detectors_{epoch}.csv")
        
        # 变异评估
        # evaluate_fewshot(epoch,dataset,self,nonself)
        # evaluate_unknown(epoch, dataset)

        # 获取数据
        data = get_data(dataset)
        # detectors = get_detectors(dataset)

        results = []
        
        # 定义测试输入
        for i, vector in enumerate(data):
            
            # 在data中随机抽10个让大模型学习空间分布
            other_indices = random.sample([idx for idx in range(len(data)) if idx != i], 10)
            input_dataset = [data[idx] for idx in other_indices] + [data[i]]
            
            # 系统提示词
            system_prompt = f"You are a professional data analyst with expertise in vector analysis and spatial distribution. Task: You will be given a dataset: {input_dataset}. Analyze the dataset's features and its spatial distribution. Based on the provided input vector, generate new {dimension}-Dimension-vectors that exhibit similar characteristics but fill in gaps in the feature space of the dataset. Each element of the new vectors must be a five-decimal number within the range (0, 1) and must not have zero as the last decimal place. Ensure that each generated number is meaningful and does not contain redundant zeros or repeating digits. Important: Ensure that the generated vectors represent meaningful variations that reflect the underlying spatial structure of the dataset. Dimension MUST be 42. Provide only the output vectors, without any explanation or process details. Output Format MUST be: 1.23456, 2.34567, 3.45678, ...Note: Focus on the spatial analysis and ensure that new vectors cover areas of the feature space that are underrepresented or not captured by the current dataset."

            # 实例化 GPTAgent
            agent = GPTAgent(
                model_name="gpt-4o-mini",  # 替换为实际模型名称
                system_prompt=system_prompt,
                task="Data Synthesis ",
                max_new_tokens=640,
                temperature=0.5,
                seed=42,
                time_sleep_min=1,
                time_sleep_max=3,
            )
            prompt = "who are you?"
            result = agent.query(prompt, print_prompt=False)
            print(result)


NameError: name 'Round' is not defined

In [ ]:
            prompt = f"Generate exactly {some} new {dimension}D-vectors based on the input vector: {vector}"
            result = agent.query(prompt, print_prompt=False)

            result = re.sub(r'(?<!\b0\.)\b[1-9]\d*\.\s*|[,\[\]]', '', result)
            vectors = re.findall(r'(\b\d+(?:\.\d+)?(?:[eE][+-]?\d+)?\b(?:\s+\b\d+(?:\.\d+)?(?:[eE][+-]?\d+)?\b){41})', result)
            
            # 筛除异常输出
            if(len(vectors)==0):
                continue
            
            # 检验生成质量
            check = len(vectors[0].split())
            # print(check)
            
            # 不合格再次请求
            while(check!=42):
                result = agent.query(prompt, print_prompt=False)
                result = re.sub(r'(?<!\b0\.)\b[1-9]\d*\.\s*|[,\[\]]', '', result)
                vectors = re.findall(r'(\b\d+(?:\.\d+)?(?:[eE][+-]?\d+)?\b(?:\s+\b\d+(?:\.\d+)?(?:[eE][+-]?\d+)?\b){41})', result)
                check = len(vectors[0].split())
                # print(check)
        
            for v in vectors:
                result = list(map(float, v.split()))
                # print(f"Mutated Vector: {vector}")
                results.append(result)

            df = pd.DataFrame(results, columns=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41"])
            df.to_csv(f"Round{r+1}/detectors_{epoch+1}.csv", index=False)
